In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import h3

import time
import datetime

**1 -- home detection based on h3 hexagons**

In [2]:
# parameters
h3_resolution = 10
minimum_checkins = 10

In [3]:
# check-in data
cdf = pd.read_csv("../data/Gowalla_totalCheckins.txt", sep="\t", header=None)
cdf.columns = ["user", "checkin_time", "lat", "lon", "location_id"]

In [4]:
# create an hour column
cdf["checkin_time"] = pd.to_datetime(cdf["checkin_time"])
cdf["hour"] = cdf["checkin_time"].dt.hour

In [5]:
def coords_to_point(lat, lon):
    """create Point geometry from lat, lon columns"""
    geometry = [Point(xy) for xy in zip(lon, lat)]
    return geometry

def point_geometry_to_h3(geodf, geometry_col, h3_resolution):
    """h3 from Point geometry"""
    h3_col = geodf.apply(lambda r: h3.geo_to_h3(r[geometry_col].y, r[geometry_col].x, h3_resolution), axis=1)
    return h3_col


In [6]:
# filter cdf by input parameters
cdf["nr_locations"] = cdf.groupby(["user"])["location_id"].transform("nunique")
lcdf = cdf[cdf["nr_locations"] >= minimum_checkins]

In [7]:
# construct h3 codes
start_time = time.time()
lcdf["point_geom"] = coords_to_point(lcdf["lat"], lcdf["lon"])
lcdf["h3"] = point_geometry_to_h3(lcdf, geometry_col="point_geom", h3_resolution=h3_resolution)
print("--- %s seconds ---" % round((time.time() - start_time), 3))

/var/folders/9d/8j37_fks51x11mk0_zwqsd940000gn/T/ipykernel_30853/3823937718.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lcdf["point_geom"] = coords_to_point(lcdf["lat"], lcdf["lon"])


--- 83.141 seconds ---


/var/folders/9d/8j37_fks51x11mk0_zwqsd940000gn/T/ipykernel_30853/3823937718.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lcdf["h3"] = point_geometry_to_h3(lcdf, geometry_col="point_geom", h3_resolution=h3_resolution)


In [8]:
# select the most visited h3 by user
def home_table(df, min_locations_in_h3, min_visits_in_h3):
    """function to select home h3 of users"""
    
    # variables to support selection
    h3_selection = df.groupby(["user", "h3"]).agg(
        locations_in_h3 = pd.NamedAgg("location_id", "nunique"),
        visits_in_h3 = pd.NamedAgg("location_id", "count")
        ).reset_index()

    h3_selection["max_locations_in_h3"] = h3_selection.groupby(["user"])["locations_in_h3"].transform("max")
    h3_selection["max_visits_in_h3"] = h3_selection.groupby(["user"])["visits_in_h3"].transform("max")

    # filter
    h3_selection = h3_selection[
        (h3_selection["visits_in_h3"] == h3_selection["max_visits_in_h3"]) &
        (h3_selection["max_locations_in_h3"] >= min_locations_in_h3) &
        (h3_selection["max_visits_in_h3"] >= min_visits_in_h3)
    ]
    
    return h3_selection

In [17]:
# no home hours focus
home_df = home_table(lcdf, min_locations_in_h3=1, min_visits_in_h3=5)

# export
home_df.to_csv("../outputs/home_location.csv", sep=";", index=False)

In [18]:
# filter for checkins in home hours -- same as for the twitter data
hh_lcdf = lcdf[
    (lcdf["hour"] >= 20) |
    (lcdf["hour"] <= 7)
]

home_df2 = home_table(hh_lcdf, min_locations_in_h3=1, min_visits_in_h3=5)

# export
home_df2.to_csv("../outputs/home_location_home_hours.csv", sep=";", index=False)

**add home locations to edgelist**

In [22]:
# edgelist
edf = pd.read_csv("../data/Gowalla_edges.txt", sep="\t")
edf.columns = ["user1", "user2"]

# home options
home_df = pd.read_csv("../outputs/home_location.csv", sep=";")
home_df2 = pd.read_csv("../outputs/home_location_home_hours.csv", sep=";")

In [23]:
edf

,user1,user2
0,0,2
1,0,3
2,0,4
3,0,5
4,0,6
...,...,...
1900648,196586,196539
1900649,196587,196540
1900650,196588,196540
1900651,196589,196547


In [24]:
home_df

,user,h3,locations_in_h3,visits_in_h3,max_locations_in_h3,max_visits_in_h3
0,0,8a489e346217fff,2,29,3,29
1,2,8a28309ae68ffff,10,28,22,28
2,4,8a283082aaf7fff,4,64,4,64
3,5,8a489e355667fff,1,11,2,11
4,7,8a489e346217fff,1,21,4,21
...,...,...,...,...,...,...
47265,196488,8a1f12b500a7fff,2,20,2,20
47266,196508,8a195da08737fff,1,8,1,8
47267,196528,8a64a485d88ffff,1,7,1,7
47268,196561,8a28346016c7fff,2,17,3,17


In [25]:
home_df2

,user,h3,locations_in_h3,visits_in_h3,max_locations_in_h3,max_visits_in_h3
0,0,8a489e346217fff,2,12,3,12
1,2,8a28308a831ffff,22,22,22,22
2,4,8a283082aaf7fff,3,26,3,26
3,5,8a489e355667fff,1,8,1,8
4,7,8a489e346217fff,1,7,4,7
...,...,...,...,...,...,...
27626,196353,8a1fa5350257fff,1,5,1,5
27627,196414,8a2f5b5b6a0ffff,1,9,1,9
27628,196456,8a9451db1977fff,4,7,4,7
27629,196478,8a6520db16d7fff,1,6,1,6
